<a href="https://colab.research.google.com/github/ucfilho/SwarmCantStopNow/blob/main/Conference_2021/ABCOptmi_SubPopulations_A_zero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from random import randint

In [2]:
def CalcFit(fun):
    result=0
    if(fun>=0):
      result=1/(fun+1)
    else:
      result=1+fabs(fun)
    return result

In [3]:
def function( sol):
  #Schwefel(x):
  x=sol  
  summ=0
  for i in range(len(x)):
    new=x[i]*np.sin((abs(x[i]))**0.5)
    summ=summ+new
    top=(418.9829*len(x)-summ) 
  
  return top;


In [4]:
def BestSource(GlobMin,GlobPars,Foods):
  FoodNumber=len(Foods[:,0])
  D=len(Foods[0,:])
  solution=np.zeros(D)
  for i in range(FoodNumber):
    if (f[i]<GlobMin):
      GlobMin=f[i]
      for j in range(D):
        GlobPars[j]=Foods[i,j]
  return GlobMin,GlobPars,Foods

In [5]:
def init(index,Foods,trial,f,fitness,MIN,MAX):
  D=len(Foods[0,:])
  FoodNumber=len(Foods[:,0])
  solution=np.zeros(D)
  for j in range(D):
    r=np.random.random()
    Foods[index,j]=r*(MAX[j]-MIN[j])+MIN[j]
    solution[j]=Foods[index,j]
    
  f[index]=function(solution)
  fitness[index]=CalcFit(f[index])
  trial[index]=0
  
  return Foods,trial,f,fitness

In [6]:
def initial(fitness,trial,f,Foods,GlobMin,GlobPars,MIN,MAX):
  D=len(Foods[0,:])
  FoodNumber=len(Foods[:,0])
  for i in range(FoodNumber):
    Foods,trial,f,fitness=init(i,Foods,trial,f,fitness,MIN,MAX) 
  GlobMin=f[0]
  for i in range(D):
    GlobPars[i]=Foods[0,i]
  return  f,Foods,GlobMin,GlobPars


In [7]:
def EmployedBees(trial,Foods,MIN,MAX):
  FoodNumber=len(Foods[:,0])
  NP=FoodNumber
  D=len(Foods[0,:])
  solution=np.zeros(D)
  for i in range(FoodNumber):
    r = np.random.random()
    par2chan= int(r*D)
    r = np.random.random()
    neighbour=int(r*D)
    if(neighbour >= NP):
      neighbour=NP-1
      
    while(neighbour==i):
      r = np.random.random()
      neighbour=int(r*FoodNumber)
      if(neighbour >= NP):
        neighbour=NP-1
    for j in range(D):
      solution[j]=Foods[i,j]

    r = np.random.random()
    A=Foods[i,par2chan]
    B=Foods[neighbour,par2chan]
    solution[par2chan]=Foods[i,par2chan]+(A-B)*(r-0.5)*2;
    if (solution[par2chan]<MIN[par2chan]):
      solution[par2chan]=MIN[par2chan]
    if (solution[par2chan]>MAX[par2chan]):
      solution[par2chan]=MAX[par2chan]
    ObjValSol=function(solution)
    FitnessSol=CalcFit(ObjValSol)
    
    if(FitnessSol>fitness[i]):
      trial[i]=0
      for j in range(D):
        Foods[i,j]=solution[j]
      f[i]=ObjValSol
      fitness[i]=FitnessSol
    else:
      trial[i]=trial[i]+1 
      
  return trial,Foods

In [8]:
def CalcProb(fitness,prob):
  FoodNumber=len(fitness)
  maxfit=fitness[0]
  for i in range(1,FoodNumber):    
    if (fitness[i]>maxfit):
      maxfit=fitness[i]
  for i in range(FoodNumber):
    prob[i]=(0.9*(fitness[i]/maxfit))+0.1
  return fitness,prob


In [9]:
def OnlookerBees(trial,Foods,MIN,MAX):
  D=len(Foods[0,:])
  solution=np.zeros(D)
  FoodNumber=len(Foods[:,0])
  i=0
  t=0
  while(t<FoodNumber):
    r = np.random.random()
    if(r<prob[i]): 
      r = np.random.random()
      t=t+1
      par2chan=int(r*D)
      r = np.random.random()
      neighbour=int(r*FoodNumber)

      while(neighbour==i):
        r = np.random.random()
        neighbour=int(r*FoodNumber)
      for j in range(D):
        solution[j]=Foods[i,j]

      r = np.random.random()
      A=Foods[i,par2chan]
      B=Foods[neighbour,par2chan]
      solution[par2chan]=A+(A-B)*(r-0.5)*2

      if (solution[par2chan]<MIN[par2chan]):
              solution[par2chan]=MIN[par2chan]
      if (solution[par2chan]>MAX[par2chan]):
              solution[par2chan]=MAX[par2chan]
      ObjValSol=function(solution)
      FitnessSol=CalcFit(ObjValSol)
              
      if(FitnessSol>fitness[i]):
        trial[i]=0
        for j in range(D):
          Foods[i,j]=solution[j]
        f[i]=ObjValSol
        fitness[i]=FitnessSol
      else:
        trial[i]=trial[i]+1
      i=i+1
      if (i==FoodNumber):
        i=0
  return trial,Foods


In [10]:
def ScoutBees(fitness,f,Foods,trial,MIN,MAX):
  FoodNumber=len(Foods[:,0])
  max_trial=0
  for i in range(1,FoodNumber):
    if (trial[i]>trial[max_trial]):
      max_trial=i
  if(trial[max_trial]>=limit):
    Foods,trial,f,fitness=init(max_trial,Foods,trial,f,fitness,MIN,MAX)
  return trial,Foods


In [11]:
'''/*Main program of the ABC algorithm*/'''

NPAR=80 #Numero de fontes de comida
ITE=50 #ITERACOES (maxCycle)
PAR=3 #NUM DE PARAMETROS A SER OTIMIZADOS
MAX=[500,500,500] # MAXIMO DE CADA PARAMETRO
MIN=[-500,-500,-500] # MINIMO DE CADA PARAMETRO

limit=100 # quantas vezes obtem resposta identica antes de encerrar 
runtime=4 # quantas vezes vai rodar para tirar a media


In [12]:

f=np.zeros(NPAR) 
Foods=np.zeros((NPAR,PAR)) 
solution=np.zeros(PAR)
fitness=np.zeros(NPAR)
trial=np.zeros(NPAR)
prob=np.zeros(NPAR)
GlobPars=np.zeros(PAR)
GlobMins=np.zeros(runtime)
GlobMin=1e99


In [13]:
# rule half-half
N_2 = int(NPAR/2)

In [14]:
f1=f[:N_2];f2 = f[N_2:];
Foods1=Foods[:N_2,:];Foods2=Foods[N_2:,:];
solution1= np.zeros(PAR);solution2= np.zeros(PAR)
fitness1=fitness[:N_2];fitness2=fitness[N_2:];
trial1=trial[:N_2];trial2=trial[N_2:]
prob1=prob[:N_2];prob2=prob[N_2:];
GlobPars1=np.zeros(PAR);GlobPars2=np.zeros(PAR);
GlobMins1=np.zeros(runtime);GlobMins2=np.zeros(runtime);
GlobMin1=1e99;GlobMin2=1e99

In [15]:


for run in range(runtime):
  f1,Foods1,GlobMin1,GlobPars1= initial(fitness1,trial1,f1,Foods1,GlobMin1,GlobPars1,MIN,MAX)
  GlobMin1,GlobPars1,Foods1=BestSource(GlobMin1,GlobPars1,Foods1)

  f2,Foods2,GlobMin2,GlobPars2= initial(fitness2,trial2,f2,Foods2,GlobMin2,GlobPars2,MIN,MAX)
  GlobMin2,GlobPars2,Foods2=BestSource(GlobMin2,GlobPars2,Foods2)

  for iter in range(ITE):
    trial1,Foods1=EmployedBees(trial1,Foods1,MIN,MAX);
    trial2,Foods2=EmployedBees(trial2,Foods2,MIN,MAX)

    fitness1,prob1=CalcProb(fitness1,prob1);
    fitness2,prob2=CalcProb(fitness2,prob2)

    trial1,Foods1=OnlookerBees(trial1,Foods1,MIN,MAX);
    trial2,Foods2=OnlookerBees(trial2,Foods2,MIN,MAX)

    GlobMin1,GlobPars1,Foods1=BestSource(GlobMin1,GlobPars1,Foods1);
    GlobMin2,GlobPars2,Foods2=BestSource(GlobMin2,GlobPars2,Foods2)

    trial1,Foods1=ScoutBees(fitness1,f1,Foods1,trial1,MIN,MAX)
    trial2,Foods2=ScoutBees(fitness2,f2,Foods2,trial2,MIN,MAX)
    G

  for j in range(PAR):
    print("GlobalParam[",j+1,"]:", GlobPars2[j])
    
  print("run",run+1,"GBest=",GlobMin2," \n");
  GlobMins2[run]=GlobMin2

mean=np.average(GlobMins2)


print("Means of",runtime,"runs:",mean,"\n")

GlobalParam[ 1 ]: 162.90354281556483
GlobalParam[ 2 ]: 467.76517970310283
GlobalParam[ 3 ]: -266.98431741464844
run 1 GBest= 0.10433268796509765  

GlobalParam[ 1 ]: -256.3802228563321
GlobalParam[ 2 ]: -500.0
GlobalParam[ 3 ]: 220.61446355886764
run 2 GBest= 0.07249743701686384  

GlobalParam[ 1 ]: -500.0
GlobalParam[ 2 ]: 353.10255284316423
GlobalParam[ 3 ]: -358.29696298181824
run 3 GBest= 0.1101219801012121  

GlobalParam[ 1 ]: -203.63645901853917
GlobalParam[ 2 ]: 10.293189517894348
GlobalParam[ 3 ]: -161.52386330773885
run 4 GBest= 0.23835021455852257  

Means of 4 runs: 0.13132557991042404 



In [20]:
Foods1[0,:]

array([424.97607546, 419.13983531, 420.97166074])

In [16]:
'''
ITE=300 #ITERACOES (maxCycle)


GlobalParam[ 1 ]: 420.96874614718286
GlobalParam[ 2 ]: 420.9687461719512
GlobalParam[ 3 ]: 420.9687463768404
run 1 GBest= 3.818269851763034e-05  

GlobalParam[ 1 ]: 420.9687462577133
GlobalParam[ 2 ]: 420.9687461387745
GlobalParam[ 3 ]: 420.9687457381019
run 2 GBest= 3.818269851763034e-05  

GlobalParam[ 1 ]: 420.9687462047169
GlobalParam[ 2 ]: 420.9687461691041
GlobalParam[ 3 ]: 420.96874633691976
run 3 GBest= 3.818269851763034e-05  

GlobalParam[ 1 ]: 420.9687464772267
GlobalParam[ 2 ]: 420.96874616214797
GlobalParam[ 3 ]: 420.9687461513788
run 4 GBest= 3.818269851763034e-05  

Means of 4 runs: 3.818269851763034e-05 
'''

'\nITE=300 #ITERACOES (maxCycle)\n\n\nGlobalParam[ 1 ]: 420.96874614718286\nGlobalParam[ 2 ]: 420.9687461719512\nGlobalParam[ 3 ]: 420.9687463768404\nrun 1 GBest= 3.818269851763034e-05  \n\nGlobalParam[ 1 ]: 420.9687462577133\nGlobalParam[ 2 ]: 420.9687461387745\nGlobalParam[ 3 ]: 420.9687457381019\nrun 2 GBest= 3.818269851763034e-05  \n\nGlobalParam[ 1 ]: 420.9687462047169\nGlobalParam[ 2 ]: 420.9687461691041\nGlobalParam[ 3 ]: 420.96874633691976\nrun 3 GBest= 3.818269851763034e-05  \n\nGlobalParam[ 1 ]: 420.9687464772267\nGlobalParam[ 2 ]: 420.96874616214797\nGlobalParam[ 3 ]: 420.9687461513788\nrun 4 GBest= 3.818269851763034e-05  \n\nMeans of 4 runs: 3.818269851763034e-05 \n'